In [10]:
# Importando libraries

import selenium
from selenium import webdriver
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager

import requests
import os
from pathlib import Path
import glob

import pandas as pd
import time
import xlrd


In [11]:
def get_file_dataframe():
    
    list_of_files = glob.glob(f'{Path.home()}\Downloads\*.xls')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    filename = latest_file

    
    try:
        #print('aqui')
        workbook =  xlrd.open_workbook_xls(filename, ignore_workbook_corruption=True)
        df = pd.read_excel(workbook,sheet_name = 0)
        #print('dentro do try')
        return df
    except:
        return print('DEU RUIM')
    
def delete_files_used(times):
    
    for i in range(times):
        
        list_of_files = glob.glob(f'{Path.home()}/Downloads/*.xls')
        latest_file = max(list_of_files, key=os.path.getctime)
        
        os.remove(latest_file)
        
    return True;

def download_wait(path_to_downloads):
    
    seconds = 0
    dl_wait = True
    
    while dl_wait and seconds < 20:
        
        time.sleep(1)
        dl_wait = False
        
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.crdownload'):
                dl_wait = True
                
        seconds += 1
        
    return seconds

In [12]:
ind_cotton_oito_dias_xpath_list = [
    '//*[@id="imagenet-content"]/div[2]/div[2]/div[1]/div[3]/a[4]',
    '//*[@id="imagenet-content"]/div[3]/div[2]/div/a[2]'
]

ind_cotton_vista_xpath_list = [
    '//*[@id="imagenet-content"]/div[2]/div[2]/div[1]/div[3]/a[4]',
    '//*[@id="imagenet-content"]/div[3]/div[2]/div/a[3]'
]

ind_cotton_quinze_dias_xpath_list =  [ 
    '//*[@id="imagenet-content"]/div[2]/div[2]/div[1]/div[3]/a[4]',
    '//*[@id="imagenet-content"]/div[3]/div[2]/div/a[4]'
]

ind_cotton_trinta_dias_xpath_list = ['//*[@id="imagenet-content"]/div[2]/div[2]/div[1]/div[3]/a[4]'
                                    ]



In [13]:
dados_dict = {
    'ind-cotton-oito-dias'        : ind_cotton_oito_dias_xpath_list ,
    'ind-cotton-vista' : ind_cotton_vista_xpath_list ,
    'ind-cotton-quinze-dias'     : ind_cotton_quinze_dias_xpath_list     ,
    'ind-cotton-trinta-dias'    : ind_cotton_trinta_dias_xpath_list    ,
}


In [14]:

ind_cotton_oito_dias_code_list   = ['algodao-pluma-oito-dias']
ind_cotton_vista_code_list       = ['algodao-pluma-vista'] 
ind_cotton_quinze_dias_code_list = ['algodao-pluma-quinze-dias' ]
ind_cotton_trinta_dias_code_list = ['algodao-pluma-trinta-dias']


In [15]:
encoder_dict =  {
    'ind-cotton-oito-dias'        : ind_cotton_oito_dias_code_list        ,
    'ind-cotton-vista' : ind_cotton_vista_code_list ,
    'ind-cotton-quinze-dias'     : ind_cotton_quinze_dias_code_list     ,
    'ind-cotton-trinta-dias'    : ind_cotton_trinta_dias_code_list         
}

In [16]:
# setup-selenium


driver = webdriver.Edge(executable_path = 'C:\\Users\\Downloads\\edgedriver_win64\\msedgedriver.exe');
# Check if the current version of chromedriver exists
# and if it doesn't exist, download it automatically,
# then add chromedriver to path
driver.maximize_window()

C:\Users\guilh\AppData\Local\Temp\ipykernel_5152\4013497868.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path = 'C:\\Users\\Downloads\\edgedriver_win64\\msedgedriver.exe');


In [17]:
url = 'https://www.cepea.esalq.usp.br/br/indicador/algodao.aspx'
driver.get(url)

data_dict = {}


In [18]:
for keys, values in encoder_dict.items():
    
    lista_xpath = dados_dict[keys]
    url = lista_xpath[0]
    
    driver.find_element('xpath',url).click()
    
    list_df = lista_xpath[1:]

    for df_xpath,encoder in zip(list_df,values):
        
        
        name = keys+'-'+encoder+'-sp_brl_close_d1.json'
        
        driver.find_element('xpath',df_xpath).click()
        download_wait(f'{Path.home()}/Downloads/')
        df = get_file_dataframe()
        data_dict[name] = df
        delete_files_used(1)
        
    
    
    

In [19]:
for values in data_dict.values():
    values.drop(index = values.index[0:3],axis = 0,inplace = True)
    values.drop(columns = values.columns[-1],axis =1,inplace = True)
    values.rename(columns = {values.columns[0]:'ano',values.columns[-1]:'valor-brl'},inplace = True)

In [20]:
base_path = 'C:\\Users\\guilh\\TM\\var_comodity\\JSONS\\CEPEA\\'
for name, df in data_dict.items():
    full_path = base_path+name
    if not df.empty:
    #    print(df.empty)
        df.to_json(full_path,indent =1)
    else:
        pass


In [21]:
driver.close()